In [16]:
%config IPCompleter.greedy=True
import numpy as np
import tflearn
from tflearn.data_utils import to_categorical,pad_sequences
from tflearn.datasets import imdb
import pandas as pd

In [2]:
# Load the data
train,test,_=imdb.load_data(path='imdb.pkl',n_words=10000,valid_portion=0.1)

In [3]:
print(train[0][0])
print(len(test))

[17, 25, 10, 406, 26, 14, 56, 61, 62, 323, 4]
2


In [4]:
trainX,trainY=train
testX,testY=test
print(trainX[:1])
print(testX[:5])


[[17, 25, 10, 406, 26, 14, 56, 61, 62, 323, 4]]
[[70, 922, 87, 394, 25, 3, 129, 883, 53, 457, 86, 879, 87, 70, 297, 42, 41, 86, 1752, 14, 40], [2, 118, 35, 6186, 5, 2, 242, 10, 397, 4, 14, 20, 6, 456, 7, 2, 1938, 7, 1, 5, 1, 4], [4014, 4, 30, 63, 32, 68, 112, 14, 508, 2, 101, 313, 246, 4, 5, 17, 10, 605, 47, 6, 666, 1749, 2783, 350, 40, 40, 360], [100, 25, 87, 277, 2, 241, 87, 1, 581, 87, 19, 42, 247, 19, 4, 17, 2444, 9001, 65, 30, 37, 427, 270, 18, 330, 290, 4], [6, 25, 134, 7844, 72, 44, 2, 151, 132, 6, 1341, 1, 1, 44, 3691, 61, 351, 1618, 20, 9607, 58, 47, 506, 52, 1, 4, 113, 406, 23, 717, 4]]


In [32]:
# Preprocess the data

trainX=pad_sequences(trainX,maxlen=100,value=0.)
testX=pad_sequences(testX,maxlen=100,value=0.)
print(len(testX))

2500


In [6]:
# Converting labels to binary vectors
print(trainY[:5])
print(testY[:5])
trainY=to_categorical(trainY,nb_classes=2)
print(type(trainY))
print(trainY[:5])
testY=to_categorical(testY,nb_classes=2)
print(testY[:5])

[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
<class 'numpy.ndarray'>
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [7]:
# Networking build

# Input layer
net=tflearn.input_data([None,100])
# Embedding layer
net=tflearn.embedding(net,input_dim=10000,output_dim=128)
net=tflearn.lstm(net,128,dropout=0.75)
net=tflearn.fully_connected(net,128,activation='ReLU')
net=tflearn.dropout(net,0.75)
net=tflearn.fully_connected(net,2,activation='softmax')
net=tflearn.regression(net,optimizer='adam',learning_rate=0.0001,loss='categorical_crossentropy')

model=tflearn.DNN(net,tensorboard_verbose=0)


In [8]:
# Training
model.fit(trainX,trainY,validation_set=0.2,show_metric=True,batch_size=32,n_epoch=10)

Training Step: 5629  | total loss: 0.16720 | time: 53.659s
| Adam | epoch: 010 | loss: 0.16720 - acc: 0.9516 -- iter: 17984/18000
Training Step: 5630  | total loss: 0.15361 | time: 57.026s
| Adam | epoch: 010 | loss: 0.15361 - acc: 0.9565 | val_loss: 0.67178 - val_acc: 0.7884 -- iter: 18000/18000
--


In [39]:
predictions=model.predict(testX)
predictions=(np.array(predictions)[:,0]>=0.5).astype(np.int_)
test_accuracy=np.mean(predictions==testY[:,0],axis=0)
print('Test Accuracy:',test_accuracy)

Test Accuracy: 0.7976
